### Problem 1
Read the noisy text data from the datafile [typos20.data](./typos20.data).
- Estimate output probabilities
- Estimate transition probabilities

Use Laplace smoothing.

Implement Viterbi algorithm for HMM. Find the most likely original text.

In [1]:
import numpy as np

In [2]:
original, typo = [], []

with open('typos20.data') as fin:
    for line in fin:
        original.append(line[0])
        typo.append(line[2])

In [3]:
c2i = {}
i2c = {}
i = 0
for c in original:
    if c not in c2i:
        c2i[c] = i
        i2c[i] = c
        i += 1
for c in typo:
    if c not in c2i:
        c2i[c] = i
        i2c[i] = c
        i += 1

In [5]:
N = len(c2i)
A = np.ones((N, N))
B = np.ones((N, N))

for i in range(len(original)-1):
    q1 = c2i[original[i]]
    q2 = c2i[original[i+1]]
    A[q1,q2] += 1

for i in range(len(original)):
    q1 = c2i[original[i]]
    o1 = c2i[typo[i]]
    B[q1,o1] += 1
    
A = A/(A.sum(axis=1)+1)
B = B/(B.sum(axis=1)+1)

In [8]:
pi = np.ones(N)/N

In [9]:
T = len(typo)

P = np.zeros((N, T))
Q = (-1) * np.ones((N, T))

o0 = c2i[typo[0]]
for i in range(N):
    P[i, 0] = np.log(pi[i]) + np.log(B[i, o0])
    Q[i, 0] = i
    
for t in range(1, T):
    ot = c2i[typo[t]]
    for i in range(N):
        pr = np.zeros(N)
        for j in range(N):
            pr[j] = P[j, t-1] + np.log(A[j, i]) + np.log(B[i, ot])
        P[i, t] = np.max(pr)
        Q[i, t] = np.argmax(pr)

In [10]:
restored = []
q = int(np.argmax(P[:,T-1]))

restored.append(i2c[q])

for t in range(T-1, 0, -1):
    q = int(Q[q, t])
    restored.append(i2c[q])
    
restored.reverse()

In [12]:
errors = 0
for i in range(len(original)):
    if original[i] != typo[i]:
        errors += 1
print(errors/len(original))

errors = 0
for i in range(len(restored)):
    if original[i] != restored[i]:
        errors += 1
print(errors/len(restored))

0.16530144439227817
0.12049490458173383
